In [6]:
# AlphaGo notebook
import sgf
import numpy as np
from openTSNE import TSNE
#from openTSNE.callbacks import ErrorLogger
from sgfmill import boards
from sgfmill import ascii_boards
from sgfmill import sgf_moves
from sgfmill import sgf as sgf2
from matplotlib import pyplot as plt
import os
from scipy import interpolate
import pywt
from scipy.fftpack import fft, ifft, fft2
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import pandas as pd


In [7]:
delete_duplicates = True

class GoBoard:
    def __init__(self):
        # create board as dictionary
        self.board = {}
        self.sgf = boards.Board(19)
        self.moves = []

        
    def encode(self, value):
        #if value == None:
        #    return [ 1, 0, 0 ]
        #if value == 'n':
        #    return [ 1, 0, 0 ]
        #if value == 'b':
        #    return [ 0, 1, 0 ]
        #if value == 'w':
        #    return [ 0, 0, 1 ]
        if value == None:
            return [ 1, 0, 0 ]
        if value == 'n':
            return [ 1, 0, 0 ]
        if value == 'b':
            return [ 0, 0, 1 ]
        if value == 'w':
            return [ 0, 1, 0 ]
        
    def encodeHaar(self, value):
        if value == None:
            return 0
        if value == 'n':
            return 0
        if value == 'b':
            return 1
        if value == 'w':
            return 1
        
        
    def histo(self, value):
        if value == None:
            return 0
        if value == 'n':
            return 0
        if value == 'b':
            return 1
        if value == 'w':
            return 1
        
        
    def toFeatureVector(self):
        vector = []
        for x in range(0, 19):
            for y in range(0, 19):
                key = chr(ord('a') + x) + chr(ord('a') + y)
                col = ord(key[0]) - ord('a')
                row = 18 - (ord(key[1]) - ord('a'))
                if (self.sgf.get(row, col) != None):
                    vector = vector + self.encode(self.sgf.get(row, col))
                else:
                    vector = vector + self.encode('n')
                
                
                #if key in self.board:
                 #   vector = vector + self.encode(self.board[key])
                #else:
                 #   vector = vector + self.encode('n')
        return vector
    
    def toFeatureVectorS(self):
        vec = []
        for x in range(0, 2):
            for y in range(0, 2):
                minX = x * 10
                minY = y * 10
                maxX = min(19, (x + 1) * 10)
                maxY = min(19, (y + 1) * 10)
                
                s = np.array([ 0, 0, 0 ])
                for i in range(minX, maxX):
                    for j in range(minY, maxY):
                        s = s + np.array(self.encode(self.sgf.get(i, j)))
                vec = vec + s.tolist()

        return vec
    
    def wavelet(self):
        vector = np.zeros((19, 19), dtype=np.float64)
        for x in range(0, 19):
            for y in range(0, 19):
                key = chr(ord('a') + x) + chr(ord('a') + y)
                col = ord(key[0]) - ord('a')
                row = 18 - (ord(key[1]) - ord('a'))
                
                if (self.sgf.get(row, col) != None):
                    vector[x, y] = self.encodeHaar(self.sgf.get(row, col))
                    
        return np.abs(fft2(vector)).flatten()
                
    
    def performMove(self, position, player):
        self.board[position] = player
        col = ord(position[0]) - ord('a')
        row = 18 - (ord(position[1]) - ord('a'))
        
        
        self.sgf.play(row, col, player.lower())
    
    
    def histogram(self):
        vector = np.zeros(38, dtype=np.float64)
        for x in range(0, 19):
            xsum = 0
            ysum = 0
            for y in range(0, 19):
                key = chr(ord('a') + x) + chr(ord('a') + y)
                col = ord(key[0]) - ord('a')
                row = 18 - (ord(key[1]) - ord('a'))
                xsum = xsum + self.histo(self.sgf.get(row, col))
                ysum = ysum + self.histo(self.sgf.get(col, row))
            vector[x] = xsum
            vector[x + 19] = ysum
        return vector
    
    
    def slidingWindow(self):
        result = np.zeros(100)
        
        for r in range(0, 100):
            i = len(self.moves) - (99 - r)
            if i >= 0 and i < len(self.moves):
                #print(f"result{r} = moves{i}")
                result[r] = self.moves[i]
                
                
        return result.tolist()
    
    def moveList(self):
        res = np.array(self.moves)
        res.resize(1000)        
            
        cA, cD = pywt.dwt(res, 'db4')
        
        return res.tolist()

        
    def play(self, move):
        if (move[1] != None):
            self.sgf.play(move[1][0], move[1][1], move[0])
            self.moves.append(move[1][0])
            self.moves.append(move[1][1])
        
    def toVector(self):
        vector = []
        for x in range(0, 19):
            for y in range(0, 19):
                key = chr(ord('a') + x) + chr(ord('a') + y)
                col = ord(key[0]) - ord('a')
                row = 18 - (ord(key[1]) - ord('a'))
                if (self.sgf.get(row, col) != None):
                    vector = vector + [self.sgf.get(row, col)]
                else:
                    vector = vector + ['n']
                

        return vector

In [18]:
encoded = []
games = []

class GG:
    def __init__(self, nodes, algo, line):
        self.nodes = nodes
        self.algo = algo
        self.line = line

#algos = ['The 2nd World AI Go Open 2018', 'The 1st World AI Go Open 2017','2020 World GO AI Championship', '2019 China Securities Cup World AI WEIQI Open', '2016 Fineart'] 
algos = ['Human vs AI', 'Human vs Human'] # Considered games

for algo in algos:
    print(f"{algo}")
    files = [f for f in os.listdir(f"{algo}")]
    print(files)
    for filename in files:
        print(filename)
        with open(f"{algo}/{filename}") as f:
            content = f.read()
            collection = sgf.parse(content)
            sgf2game = sgf2.Sgf_game.from_string(content)
            movelist = sgf_moves.get_setup_and_moves(sgf2game)

            g = []
            board = GoBoard()
            for node in movelist[1]:
                if (node[1] != None):
                    board.play(node)
                    g.append(board.toVector())
                  
                 
                    encoded.append(board.histogram())

            sgfboard = boards.Board(19)

            line = filename
            if line.startswith('Game'):
                line = f"{algo} {line}"
            games.append(GG(g, algo, line))


if delete_duplicates:
    encoded, indices, counts = np.unique(encoded, axis=0, return_inverse=True, return_counts=True)

Human vs AI
['AlphaGo  vs Chen_Yaoye_&_Zhou_Ruiyang.sgf', 'Park Junghwan vs AlphaGo.sgf', 'Lee_Sedol vs AlphaGo .sgf', 'Alphago vs Yang.sgf', 'Chang Hao vs AlphaGo.sgf', 'Ke Jie vs AlphaGo.sgf', 'Fan_Hui vs AlphgaGo.sgf', 'Gu Li vs AlphaGo.sgf', 'AlpghaGo vs Shin Jinseo.sgf', 'AlphaGo vs AlphaGo.sgf']
AlphaGo  vs Chen_Yaoye_&_Zhou_Ruiyang.sgf
Park Junghwan vs AlphaGo.sgf
Lee_Sedol vs AlphaGo .sgf
Alphago vs Yang.sgf
Chang Hao vs AlphaGo.sgf
Ke Jie vs AlphaGo.sgf
Fan_Hui vs AlphgaGo.sgf
Gu Li vs AlphaGo.sgf
AlpghaGo vs Shin Jinseo.sgf
AlphaGo vs AlphaGo.sgf
Human vs Human
['Cho U vs Ichiriki Ryo.sgf', 'Kono Rin vs Cho chikun.sgf', 'Shibano Toramaru vs Ke Jie.sgf', 'Lee seedol vs Gu Li.sgf', 'Gu Li vs Cho Hangseung.sgf', 'Lee Seedol vs Chang Hao.sgf', 'Kyo Kagen vs Iyama Yuta.sgf', 'Baek Hongsuk vs Dang Yifei.sgf', 'Mutsuara Yuta vs Takao Shinji.sgf', 'HIrarata Tomaya vs Iyama Yuta.sgf']
Cho U vs Ichiriki Ryo.sgf
Kono Rin vs Cho chikun.sgf
Shibano Toramaru vs Ke Jie.sgf
Lee seedol vs Gu 

In [19]:
print(len(games))

20


In [20]:
csv = open("go_data.csv", "w")
csv.write("index,state_type,competitors,competetion")
for x in range(0, 19):
    for y in range(0, 19):
        key = chr(ord('a') + x) + chr(ord('a') + y)
        csv.write(',')
        csv.write(key)
csv.write("\n")
idx = 0


for i, game in enumerate(games):
    for i, state in enumerate(game.nodes):
        csv.write(str(i))
        csv.write(',')
        if i == 0 :
            csv.write('start')
            
        elif len(game.nodes)  == i + 1:
            csv.write('end')

        else:
            csv.write('intermediate')

        csv.write(',')
        
        csv.write(game.line)
        csv.write(',')
        csv.write(game.algo)
        idx = idx + 1
        for fa in state:
            csv.write(',')
            csv.write(str(fa))
        csv.write('\n')
csv.close()